In [95]:
from sklearn.model_selection import LeaveOneOut
import networkx as nx
import math
import sys
import pandas as pd

In [96]:
def initialize_scores(G):
    fairness = {}
    goodness = {}
    
    nodes = G.nodes()
    #print(nodes)
    for node in nodes:
        fairness[node] = 1
        try:
            goodness[node] = G.in_degree(node, weight='weight')*1.0/G.in_degree(node)
        except:
            goodness[node] = 0
    return fairness, goodness

In [97]:
def compute_fairness_goodness(G):
    fairness, goodness = initialize_scores(G)
    
    nodes = G.nodes()
    iter = 0
    while iter < 100:
        df = 0
        dg = 0

        print('-----------------')
        print("Iteration number", iter)
        
        print('Updating goodness')
        for node in nodes:
            inedges = G.in_edges(node, data='weight')
            #print(inedges)
            g = 0
            for edge in inedges:
                g += fairness[edge[0]]*edge[2]

            try:
                dg += abs(g/len(inedges) - goodness[node])
                goodness[node] = g/len(inedges)
            except:
                pass

        print('Updating fairness')
        for node in nodes:
            outedges = G.out_edges(node, data='weight')
            f = 0
            for edge in outedges:
                f += 1.0 - abs(edge[2] - goodness[edge[1]])/2.0
            try:
                df += abs(f/len(outedges) - fairness[node])
                fairness[node] = f/len(outedges)
            except:
                pass
        
        print('Differences in fairness score and goodness score = %.2f, %.2f' % (df, dg))
        if df < math.pow(10, -6) and dg < math.pow(10, -6):
            break
        iter+=1
    
    return fairness, goodness

In [98]:
G = nx.DiGraph()

In [99]:
f = open("OTCNet.csv","r")

In [100]:
for l in f:
    try:
        ls = l.strip().split(",")
        G.add_edge(ls[0], ls[1], weight = float(ls[2])) ## the weight should already be in the range of -1 to 1
    except:
        continue
f.close()

In [101]:
fairness, goodness = compute_fairness_goodness(G)

-----------------
Iteration number 0
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 382.21, 0.00
-----------------
Iteration number 1
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 34.47, 102.29
-----------------
Iteration number 2
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 2.39, 6.97
-----------------
Iteration number 3
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 0.39, 1.06
-----------------
Iteration number 4
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 0.10, 0.24
-----------------
Iteration number 5
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 0.03, 0.06
-----------------
Iteration number 6
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 0.01, 0.02
-----------------
Iteration number 7
Updating good

In [102]:
import pandas as pd
df_f = pd.read_csv('./OTCNet.csv')
df = pd.DataFrame()

df['Id'] = df_f[['node1', 'node2']].apply(lambda x: '-'.join(x.fillna('').map(str)), axis=1)
df['Id'] = df['Id'].str.strip('-')

result_f = {}
for index, row in df_f.iterrows():
    node = str(int(row['node1']))
    result_f[index] = fairness[node]
    
result_g = {}
for index, row in df_f.iterrows():
    node = str(int(row['node2']))
    result_g[index] = goodness[node]

In [103]:
df['fairness'] = pd.Series(result_f, index=df_f.index, dtype=float)
df['goodness'] = pd.Series(result_g, index=df_f.index, dtype=float)

In [108]:
df['rating'] = pd.Series(df_f['rating'], index=df_bd.index, dtype=float)

In [109]:
df

,Id,fairness,goodness,rating
0,6-2,0.895726,0.269531,0.4
1,6-5,0.895726,0.214168,0.2
2,1-15,0.922436,0.144465,0.1
3,4-3,0.891287,-0.035796,0.7
4,13-16,0.945563,0.756450,0.8
5,13-10,0.945563,0.552062,0.8
6,7-5,0.943844,0.214168,0.1
7,2-21,0.893744,0.224765,0.5
8,2-20,0.893744,0.182410,0.5
9,21-2,0.880717,0.269531,0.5


In [110]:
df.to_csv('./fg_result.csv', index=False)